In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib
print("OK ✅", pd.__version__, sklearn.__version__)

OK ✅ 2.3.3 1.6.1


# Laboratorio 1 — AlpesHearth

## 1. Carga de datos

In [ ]:
import pandas as pd

TRAIN_PATH = "Datos Lab 1.csv"
TEST_PATH  = "Datos Test Lab 1.csv"       
TARGET_COL = "CVD Risk Score"    

df = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH, sep=";")

print("Train:", df.shape)
print("Test:", df_test.shape)
df.head()


Train: (1639, 24)
Test: (194, 22)


,Patient ID,Date of Service,Sex,Age,Weight (kg),Height (m),BMI,Abdominal Circumference (cm),Blood Pressure (mmHg),Total Cholesterol (mg/dL),...,Physical Activity Level,Family History of CVD,Height (cm),Waist-to-Height Ratio,Systolic BP,Diastolic BP,Blood Pressure Category,Estimated LDL (mg/dL),CVD Risk Score,CVD Risk Level
0,isDx5313,"November 08, 2023",M,44.0,114.300,1.720,38.600,100.000,112/83,228.0,...,High,N,172.000,0.581,112.0,83.0,Hypertension Stage 1,121.0,19.880,HIGH
1,LHCK2961,20/03/2024,F,57.0,92.923,1.842,33.116,106.315,101/91,158.0,...,High,Y,184.172,0.577,101.0,91.0,Hypertension Stage 2,57.0,16.833,INTERMEDIARY
2,WjVn1699,2021-05-27,F,NaN,73.400,1.650,27.000,78.100,90/74,135.0,...,High,N,165.000,0.473,90.0,74.0,Normal,45.0,12.600,LOW
3,dCDO1109,"April 18, 2022",F,35.0,113.300,1.780,35.800,79.600,92/89,158.0,...,Moderate,Y,178.000,0.447,92.0,89.0,Hypertension Stage 1,94.0,14.920,HIGH
4,pnpE1080,01/11/2024,F,48.0,102.200,1.750,33.400,106.700,121/68,207.0,...,Low,Y,175.000,0.610,121.0,68.0,Elevated,128.0,18.870,HIGH


## 2. Exploración y perfilamiento

## 3. Preparación de datos

## 4. Modelos (2 pipelines)

## 5. Tabla comparativa (RMSE/MAE/R2)

## 6. Interpretación e importancia de variables

## 7. Respuestas “Análisis de resultados”

## 8. Predicciones en test no etiquetado

## 9. Uso de herramientas de IA generativa